In [6]:
"""
1. Setup and Configuration

Load required libraries and set working directory.
"""
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

# Configuration
workdir = "~/Desktop/work/protein_linkers/input_2"
workdir = os.path.expanduser(workdir)
elm_csv_path = os.path.join(workdir, 'elm', 'elm_results.csv')
fasta_path = os.path.join(workdir, 'proteins.fa')

In [7]:
"""
2. Load Linker + Domain Data

Load the three clustered linker dictionaries (short, medium, long) that were
generated by the clustering analysis, plus the filtered proteins dictionary
with all domains and linkers.
"""

# Load short linkers
short_linkers_path = os.path.join(workdir, 'short_linkers.json')
with open(short_linkers_path, 'r') as f:
    short_linkers = json.load(f)

# Load medium linkers
medium_linkers_path = os.path.join(workdir, 'medium_linkers.json')
with open(medium_linkers_path, 'r') as f:
    medium_linkers = json.load(f)

# Load long linkers
long_linkers_path = os.path.join(workdir, 'long_linkers.json')
with open(long_linkers_path, 'r') as f:
    long_linkers = json.load(f)

# Load filtered proteins (contains all domains and linkers after outlier removal)
filtered_proteins_path = os.path.join(workdir, 'filtered_proteins.json')
with open(filtered_proteins_path, 'r') as f:
    filtered_proteins = json.load(f)


In [ ]:
# Check structure of the data
example_protein = list(filtered_proteins.keys())[0]
print(f"Example protein: {example_protein}\n")
print("\nProtein data structure:\n")
print(json.dumps(filtered_proteins[example_protein], indent=2)[:1000], "\n")

# Find a protein with linkers
for pid, pdata in filtered_proteins.items():
    if pdata.get('linkers'):
        print(f"\nExample protein with linkers: {pid}\n")
        print(json.dumps(pdata, indent=2)[:1500], "\n")
        break

In [9]:
"""
3. Load Protein Sequences

Load the FASTA file to extract linker sequences for ELM analysis.
"""

def load_fasta(fasta_path):
    """
    Load protein sequences from a FASTA file.

    Parameters:
    -----------
    fasta_path : str
        Path to FASTA file

    Returns:
    --------
    dict
        Dictionary with protein accessions as keys and sequences as values
    """
    sequences_dict = {}

    with open(fasta_path, 'r') as f:
        current_acc = None
        current_seq = []

        for line in f:
            line = line.strip()
            if line.startswith('>'):
                # Save previous sequence if exists
                if current_acc:
                    sequences_dict[current_acc] = ''.join(current_seq)

                # Start new sequence
                current_acc = line[1:]
                current_seq = []
            else:
                current_seq.append(line)

        # Save last sequence
        if current_acc:
            sequences_dict[current_acc] = ''.join(current_seq)

    return sequences_dict


# Load sequences
sequences_dict = load_fasta(fasta_path)

print(f"✓ Loaded {len(sequences_dict)} protein sequences from proteins.fa\n")

In [ ]:
"""
4. Load ELM Results (CSV) and Inspect Structure
"""
import csv
from IPython.display import display

# Ensure the CSV exists
if not os.path.exists(elm_csv_path):
    raise FileNotFoundError(f"ELM results CSV not found at: {elm_csv_path}")

# Try to detect delimiter robustly
with open(elm_csv_path, "r", encoding="utf-8", errors="replace") as fh:
    sample = fh.read(4096)
    try:
        dialect = csv.Sniffer().sniff(sample, delimiters=[",", "\t", ";", "|"])
        sep = dialect.delimiter
    except csv.Error:
        # Fallback: choose the most frequent candidate delimiter
        candidates = [",", "\t", ";", "|"]
        counts = {d: sample.count(d) for d in candidates}
        sep = max(counts, key=counts.get)

print(f"Detected delimiter: {repr(sep)}")

# Load CSV with a tolerant parser for occasional bad lines
elm_df = pd.read_csv(
    elm_csv_path,
    sep=sep,
    engine="python",
    on_bad_lines="warn"
)
elm_df[0:1]


In [ ]:
"""
5. Filter ELM Results by Protein Existence

Remove entries where the protein ID (extracted from region_id)
is not in filtered_proteins.keys()
"""

# Extract protein ID from region_id (format: <protein>_<domain|linker>_<idx>_<name>)
elm_df['protein_id'] = elm_df['region_id'].str.split('_').str[0]

# Before filtering
print(f"Before filtering: {len(elm_df):,} rows")

# Filter to keep only rows where protein exists in filtered_proteins
elm_df = elm_df[elm_df['protein_id'].isin(filtered_proteins.keys())]

# After filtering
print(f"After filtering: {len(elm_df):,} rows")


In [71]:
"""
6. Add ELM motifs to domains and linkers in filtered_proteins

Refactored into smaller, focused functions for better readability and maintainability.
"""

def _convert_domains_to_dicts(filtered_proteins):
    """
    Convert domain lists [name, start, end] to dictionaries with motifs field.

    Parameters:
    -----------
    filtered_proteins : dict
        Protein data with domains as lists or dicts
    """
    for pid, protein_entry in filtered_proteins.items():
        if 'domains' in protein_entry:
            new_domains = []
            for domain in protein_entry['domains']:
                if isinstance(domain, list):
                    domain_dict = {
                        'name': domain[0],
                        'start': domain[1],
                        'end': domain[2],
                        'motifs': []
                    }
                    new_domains.append(domain_dict)
                elif isinstance(domain, dict):
                    if 'motifs' not in domain:
                        domain['motifs'] = []
                    new_domains.append(domain)
            protein_entry['domains'] = new_domains


def _convert_linkers_to_dicts(filtered_proteins):
    """
    Convert linker lists [type, start, end] to dictionaries with motifs field.

    Parameters:
    -----------
    filtered_proteins : dict
        Protein data with linkers as lists or dicts
    """
    for pid, protein_entry in filtered_proteins.items():
        if 'linkers' in protein_entry:
            new_linkers = []
            for linker in protein_entry['linkers']:
                if isinstance(linker, list):
                    linker_dict = {
                        'type': linker[0],
                        'start': linker[1],
                        'end': linker[2],
                        'motifs': []
                    }
                    new_linkers.append(linker_dict)
                elif isinstance(linker, dict):
                    if 'motifs' not in linker:
                        linker['motifs'] = []
                    new_linkers.append(linker)
            protein_entry['linkers'] = new_linkers


def _parse_region_id(region_id):
    """
    Parse region_id into components.

    Parameters:
    -----------
    region_id : str
        Format: <pid>_<domain|linker>_<idx>_<name>

    Returns:
    --------
    tuple : (protein_id, idx) or (None, None) if parsing fails
    """
    parts = region_id.split('_', 3)
    if len(parts) < 3:
        return None, None

    try:
        idx = int(parts[2]) - 1  # Convert from 1-based to 0-based
        return parts[0], idx
    except ValueError:
        return None, None


def _assign_motifs_to_domains(elm_df, filtered_proteins):
    """
    Assign ELM motifs to domains.

    Parameters:
    -----------
    elm_df : DataFrame
        ELM results filtered for region_type='domains'
    filtered_proteins : dict
        Protein data with domains

    Returns:
    --------
    int : Number of motifs added
    """
    # Ensure domains are dicts
    _convert_domains_to_dicts(filtered_proteins)

    motif_count = 0
    for _, row in elm_df.iterrows():
        if row['region_type'] != 'domains':
            continue

        pid, idx = _parse_region_id(row['region_id'])
        if pid is None:
            continue

        protein_entry = filtered_proteins.get(pid)
        if not protein_entry or 'domains' not in protein_entry:
            continue

        if idx < 0 or idx >= len(protein_entry['domains']):
            continue

        # Add motif
        motif_info = {
            'motif': row['motif_id'],
            'start': int(row['motif_start']),
            'end': int(row['motif_end'])
        }
        protein_entry['domains'][idx]['motifs'].append(motif_info)
        motif_count += 1

    return motif_count


def _assign_motifs_to_linkers(elm_df, filtered_proteins, linker_type):
    """
    Assign ELM motifs to linkers of a specific type.

    Parameters:
    -----------
    elm_df : DataFrame
        ELM results
    filtered_proteins : dict
        Protein data with linkers
    linker_type : str
        One of: 'inner', 'c-terminus', 'n-terminus'

    Returns:
    --------
    int : Number of motifs added
    """
    # Ensure linkers are dicts with motifs field
    _convert_linkers_to_dicts(filtered_proteins)

    motif_count = 0
    for _, row in elm_df.iterrows():
        if row['region_type'] != linker_type:
            continue

        pid, idx = _parse_region_id(row['region_id'])
        if pid is None:
            continue

        protein_entry = filtered_proteins.get(pid)
        if not protein_entry or 'linkers' not in protein_entry:
            continue

        if idx < 0 or idx >= len(protein_entry['linkers']):
            continue

        target_linker = protein_entry['linkers'][idx]

        # Verify the linker type matches
        if not isinstance(target_linker, dict) or target_linker.get('type') != linker_type:
            continue

        # Add motif
        motif_info = {
            'motif': row['motif_id'],
            'start': int(row['motif_start']),
            'end': int(row['motif_end'])
        }
        target_linker['motifs'].append(motif_info)
        motif_count += 1

    return motif_count


def add_motifs_by_type(elm_df, filtered_proteins, region_type):
    """
    Wrapper function that routes to the appropriate assignment function based on region type.

    Parameters:
    -----------
    elm_df : DataFrame
        ELM results
    filtered_proteins : dict
        Protein data
    region_type : str
        Either 'domains' or one of the linker types: 'inner', 'c-terminus', 'n-terminus'

    Returns:
    --------
    int : Number of motifs added
    """
    if region_type == 'domains':
        return _assign_motifs_to_domains(elm_df, filtered_proteins)
    elif region_type in ['inner', 'c-terminus', 'n-terminus']:
        return _assign_motifs_to_linkers(elm_df, filtered_proteins, region_type)
    else:
        raise ValueError(f"Unknown region_type: {region_type}")


In [ ]:
# Ensure protein_id column exists
if 'protein_id' not in elm_df.columns:
    elm_df['protein_id'] = elm_df['region_id'].str.split('_').str[0]

# Process all region types
region_types = ['domains', 'inner', 'c-terminus', 'n-terminus']
total_motifs = 0

for region_type in region_types:
    print(f"Processing {region_type}...")
    count = add_motifs_by_type(elm_df, filtered_proteins, region_type)
    print(f"✓ Added {count:,} motifs to {region_type}")
    total_motifs += count

print(f"\n{'='*60}")
print(f"TOTAL: Added {total_motifs:,} motifs across all regions")
print(f"{'='*60}")

# Sort motifs by start position in each domain and linker
print("\nSorting motifs by start position...")
sorted_count = 0

for pid, protein_entry in filtered_proteins.items():
    # Sort domain motifs
    if 'domains' in protein_entry:
        for domain in protein_entry['domains']:
            if isinstance(domain, dict) and 'motifs' in domain and domain['motifs']:
                domain['motifs'].sort(key=lambda m: m['start'])
                sorted_count += len(domain['motifs'])

    # Sort linker motifs
    if 'linkers' in protein_entry:
        for linker in protein_entry['linkers']:
            if isinstance(linker, dict) and 'motifs' in linker and linker['motifs']:
                linker['motifs'].sort(key=lambda m: m['start'])
                sorted_count += len(linker['motifs'])

print(f"✓ Sorted {sorted_count:,} motifs by start position")

# Save to JSON
output_path = os.path.join(workdir, 'proteins_elm_motifs.json')

with open(output_path, 'w') as f:
    json.dump(filtered_proteins, f, indent=2)


Processing domains...
✓ Added 19,094 motifs to domains
Processing inner...
✓ Added 19,094 motifs to domains
Processing inner...
✓ Added 1,095 motifs to inner
Processing c-terminus...
✓ Added 1,095 motifs to inner
Processing c-terminus...
✓ Added 80 motifs to c-terminus
Processing n-terminus...
✓ Added 80 motifs to c-terminus
Processing n-terminus...
✓ Added 81 motifs to n-terminus

TOTAL: Added 20,350 motifs across all regions

Sorting motifs by start position...
✓ Sorted 179,382 motifs by start position
✓ Added 81 motifs to n-terminus

TOTAL: Added 20,350 motifs across all regions

Sorting motifs by start position...
✓ Sorted 179,382 motifs by start position
